In [60]:
import requests
import pandas as pd

In [56]:
# Eurostat 
# in order to build the key
# need to know the dataflow (variable): 
# https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/ESTAT/NAMA_10_GDP if we select GDP
# then the data structure (dimensions of the variable we can select) associated with the dataflow: 
# https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/dataflow/ESTAT/NAMA_10_GDP?references=children so we know in which order to declare the dimensions
# and finally the codelist associated with each structure: 
# https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/codelist/ESTAT/FREQ if we select the frequency FREQ

# quarterly variables

# Price indices of the means of agricultural production, input (2010 = 100), until 2017 only
# freq,p_adj,unit,product,geo,TIME_PERIOD 

# Price indices of the means of agricultural production, input (2015 = 100), from 2015 only
# freq,p_adj,unit,product,geo,TIME_PERIOD 


# monthly variables

# Slaughtering in slaughterhouses
# freq,meat,meatitem,unit,geo,TIME_PERIOD

# Poultry   
# freq,animals,hatchitm,geo,TIME_PERIOD


In [ ]:
#!pip install pandasdmx
import pandasdmx as sdmx
estat = sdmx.Request('ESTAT')

In [287]:
flow_msg = estat.dataflow()

In [10]:
dataflows = sdmx.to_pandas(flow_msg.dataflow)
dataflows.head()

DS-018995                          EU trade since 1999 by SITC
DS-022469    Extra-EU trade since 2000 by mode of transport...
DS-032655                     EU trade since 1988 by BEC/rev.4
DS-043227                        EFTA trade since 1995 by SITC
DS-056697    EU enlargement countries trade since 2002, by ...
dtype: object

In [53]:
agri = dataflows[dataflows.str.contains('agricultur', case=False)]
prod = dataflows[dataflows.str.contains('production', case=False)]
outcome = set(agri) & set(prod)
outcome

{'EC indices of purchase prices of the means of agricultural production (1995 = 100) - annual data',
 'EC indices of purchase prices of the means of agricultural production (1995 = 100) - monthly data',
 'Holdings, cultivated area under vines (CAV) and agricultural area in use broken down by type of production, size class and regions',
 'Main farm indicators by agricultural area, type and economic size of the farm, share of consumed production, legal status of the holding and NUTS2 region',
 'Organic crop area by agricultural production methods and crops',
 'Organic crop area by agricultural production methods and crops (until 2011)',
 'Price indices of the means of agricultural production, input (2000 = 100) - annual data',
 'Price indices of the means of agricultural production, input (2000 = 100) - monthly data',
 'Price indices of the means of agricultural production, input (2000 = 100) - quarterly data',
 'Price indices of the means of agricultural production, input (2005 = 100) -

In [64]:
take = dataflows[dataflows.str.contains('Price indices of the means of agricultural production', case=False)]
take

apri_pi00_ina    Price indices of the means of agricultural pro...
apri_pi00_inm    Price indices of the means of agricultural pro...
apri_pi00_inq    Price indices of the means of agricultural pro...
apri_pi05_ina    Price indices of the means of agricultural pro...
apri_pi05_inq    Price indices of the means of agricultural pro...
apri_pi10_ina    Price indices of the means of agricultural pro...
apri_pi10_inq    Price indices of the means of agricultural pro...
apri_pi15_ina    Price indices of the means of agricultural pro...
apri_pi15_inq    Price indices of the means of agricultural pro...
dtype: object

In [200]:
metadata = estat.datastructure('DSD_apri_pi15_inq')
metadata.codelist
for cl in 'CL_PRODUCT', 'CL_P_ADJ', 'CL_UNIT', 'CL_FREQ', 'CL_GEO':
    print(sdmx.to_pandas(metadata.codelist[cl]))

                                                         name      parent
CL_PRODUCT                                                               
200000      Goods and services currently consumed in agric...  CL_PRODUCT
201000                               Seeds and planting stock  CL_PRODUCT
202000                                     Energy, lubricants  CL_PRODUCT
202100                                            Electricity  CL_PRODUCT
202200                                      Fuels for heating  CL_PRODUCT
202300                                            Motor fuels  CL_PRODUCT
202400                                             Lubricants  CL_PRODUCT
203000                         Fertilisers and soil improvers  CL_PRODUCT
203100                                   Straight fertilizers  CL_PRODUCT
203110                                Nitrogenous fertilizers  CL_PRODUCT
203120                                 Phosphatic fertilizers  CL_PRODUCT
203130                                

In [749]:
key = dict(GEO='EU28+FR+EU27_2020', PRODUCT='200000+210000', UNIT='I15', P_ADJ='RI')
params = dict(startPeriod='2000', endPeriod='2021')
resp = estat.data('apri_pi15_inq', key=key, params=params)
data = resp.to_pandas(
    datetime={'dim': 'TIME_PERIOD', 'freq': 'FREQ'})
data

2022-12-20 01:05:21,770 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


P_ADJ              RI                                          
UNIT              I15                                          
PRODUCT        200000                    210000                
GEO         EU27_2020    EU28      FR EU27_2020    EU28      FR
TIME_PERIOD                                                    
2015Q1         100.97  101.14  100.83    100.37  100.42  100.25
2015Q2         100.16  100.32  100.77     99.32   99.40   99.79
2015Q3          99.92   99.84   99.61     99.91   99.89  100.04
2015Q4          98.96   98.71   98.78    100.41  100.31   99.92
2016Q1          98.30   98.00   97.41    101.36  101.22  100.38
2016Q2          97.01   96.91   96.42    100.42  100.34   99.55
2016Q3          96.54   96.50   95.44    100.78  100.69  100.06
2016Q4          96.40   96.55   95.91    101.13  100.99  100.43
2017Q1          98.28   98.44   97.13    101.49  101.34  100.58
2017Q2          96.84   97.03   95.90    101.07  100.81  100.91
2017Q3          96.26   96.46   95.18    101.50  101.26  101.39
2017Q4          96.76   96.97   95.83    101.95  101.70  102.04
2018Q1          98.37   98.59   96.33    102.92  102.64  102.47
2018Q2          98.13   98.44   96.62    101.76  101.73  102.23
2018Q3          98.88   99.39   97.55    102.21  102.14  102.71
2018Q4         100.23  100.75   99.08    102.39  102.28  103.65
2019Q1         100.66  101.20   98.92    103.73  103.56  103.95
2019Q2          98.99   99.59   97.99    102.59  102.44  103.07
2019Q3          98.14   98.60   96.86    102.86  102.65  102.94
2019Q4          97.60   97.95   96.62    102.97  102.71  103.07
2020Q1          98.08     NaN   96.06    104.27     NaN  103.79
2020Q2          95.75     NaN   94.28    103.94     NaN  103.48
2020Q3          95.64     NaN   94.04    104.07     NaN  103.02
2020Q4          96.76     NaN   94.97    104.75     NaN  104.19
2021Q1         100.22     NaN   98.05    105.33     NaN  105.56
2021Q2         101.96     NaN   99.77    105.12     NaN  106.01
2021Q3         104.85     NaN  102.08    106.48     NaN  108.49
2021Q4         111.97     NaN  109.21    106.85     NaN  109.77

In [292]:
# sorting by columns
price_Goods_services_consumed_agri_prod15 = data.loc[:, ('RI', 'I15', '200000')]
price_Goods_services_contributing_agri_prod15 = data.loc[:, ('RI', 'I15', '210000')]

Now for the remaining datasets

In [ ]:
key = dict(GEO='EU28+FR+EU27_2020+EU27_2007', HATCHITM='CH', ANIMALS='A5130P')
params = dict(startPeriod='2000-01', endPeriod='2021-12')
resp = estat.data('apro_ec_poulm', key=key, params=params)
data = resp.to_pandas(
    datetime={'dim': 'TIME_PERIOD', 'freq': 'FREQ'})

prod_chicken = data.loc[:, ('A5130P', 'CH')]

key = dict(GEO='EU28+FR+EU27_2020+EU27_2007', MEAT='B1000', MEATITEM='SL', UNIT='THS_T')
params = dict(startPeriod='2000-01', endPeriod='2021-12')
resp = estat.data('apro_mt_pwgtm', key=key, params=params)
data = resp.to_pandas(
    datetime={'dim': 'TIME_PERIOD', 'freq': 'FREQ'})

prod_bovine = data.loc[:, ('B1000', 'SL', 'THS_T')]

key = dict(GEO='FR', PRODUCT='200000+210000', UNIT='I00', P_ADJ='RI')
params = dict(startPeriod='2000', endPeriod='2021')
resp = estat.data('apri_pi00_inq', key=key, params=params)
data = resp.to_pandas(
    datetime={'dim': 'TIME_PERIOD', 'freq': 'FREQ'})

price_Goods_services_consumed_agri_prod00 = data.loc[:, ('RI', 'I00', '200000')]
price_Goods_services_contributing_agri_prod00 = data.loc[:, ('RI', 'I00', '210000')]

key = dict(GEO='FR', PRODUCT='200000+210000', UNIT='I05', P_ADJ='RI')
params = dict(startPeriod='2000', endPeriod='2021')
resp = estat.data('apri_pi05_inq', key=key, params=params)
data = resp.to_pandas(
    datetime={'dim': 'TIME_PERIOD', 'freq': 'FREQ'})

price_Goods_services_consumed_agri_prod05 = data.loc[:, ('RI', 'I05', '200000')]
price_Goods_services_contributing_agri_prod05 = data.loc[:, ('RI', 'I05', '210000')]

key = dict(GEO='FR', PRODUCT='200000+210000', UNIT='I10', P_ADJ='RI')
params = dict(startPeriod='2000', endPeriod='2021')
resp = estat.data('apri_pi10_inq', key=key, params=params)
data = resp.to_pandas(datetime('TIME_PERIOD'))
    
price_Goods_services_consumed_agri_prod10 = data.loc[:, ('RI', 'I10', '200000')]
price_Goods_services_contributing_agri_prod10 = data.loc[:, ('RI', 'I10', '210000')]

In [194]:
def relabelling(df,dataflow_code):
    col = list(df.columns)
    mapping = {}
    #df.reset_index()
    for i in range(len(col)):
        col_name = col[i] + "_" + dataflow_code
        mapping[df.columns[i]] = col_name
        #df.rename(columns={df.columns[i]: col_name})
    df = df.rename(columns=mapping)
    return df

In [300]:
df1 = relabelling(prod_bovine, 'prod_bovine')
df2 = relabelling(prod_chicken, 'prod_chicken')
df3 = relabelling(price_Goods_services_consumed_agri_prod00, 'price_Goods_services_consumed_agri_prod00')
df4 = relabelling(price_Goods_services_contributing_agri_prod00, 'price_Goods_services_contributing_agri_prod00')
df5 = relabelling(price_Goods_services_consumed_agri_prod05, 'price_Goods_services_consumed_agri_prod05')
df6 = relabelling(price_Goods_services_contributing_agri_prod05, 'price_Goods_services_contributing_agri_prod05')
df7 = relabelling(price_Goods_services_consumed_agri_prod10, 'price_Goods_services_consumed_agri_prod10')
df8 = relabelling(price_Goods_services_contributing_agri_prod10, 'price_Goods_services_contributing_agri_prod10')
df9 = relabelling(price_Goods_services_contributing_agri_prod15, 'price_Goods_services_contributing_agri_prod15')
df10 = relabelling(price_Goods_services_consumed_agri_prod15, 'price_Goods_services_consumed_agri_prod15')


In [421]:
#!pip install pathlib
from pathlib import Path  

In [631]:
df = relabelling(price_Goods_services_consumed_agri_prod15, 'price_Goods_services_consumed_agri_prod15')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_consumed_agri_prod15.csv')
df.to_csv(filepath, index=True)

df = relabelling(price_Goods_services_contributing_agri_prod15, 'price_Goods_services_contributing_agri_prod15')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_contributing_agri_prod15.csv')
df.to_csv(filepath, index=True)

df = relabelling(price_Goods_services_consumed_agri_prod10, 'price_Goods_services_consumed_agri_prod10')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_consumed_agri_prod10.csv')
df.to_csv(filepath, index=True)

df = relabelling(price_Goods_services_contributing_agri_prod10, 'price_Goods_services_contributing_agri_prod10')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_contributing_agri_prod10.csv')
df.to_csv(filepath, index=True)

df = relabelling(price_Goods_services_consumed_agri_prod05, 'price_Goods_services_consumed_agri_prod05')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_consumed_agri_prod05.csv')
df.to_csv(filepath, index=True)

df = relabelling(price_Goods_services_contributing_agri_prod05, 'price_Goods_services_contributing_agri_prod05')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_contributing_agri_prod05.csv')
df.to_csv(filepath, index=True)

df = relabelling(price_Goods_services_consumed_agri_prod00, 'price_Goods_services_consumed_agri_prod00')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_consumed_agri_prod00.csv')
df.to_csv(filepath, index=True)

df = relabelling(price_Goods_services_contributing_agri_prod15, 'price_Goods_services_contributing_agri_prod00')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//price_Goods_services_contributing_agri_prod00.csv')
df.to_csv(filepath, index=True)

df = relabelling(prod_chicken, 'prod_chicken')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//prod_chicken.csv')
df.to_csv(filepath, index=True)

df = relabelling(prod_bovine, 'prod_bovine')
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//prod_bovine.csv')
df.to_csv(filepath, index=True)


In [750]:
def convert_to_monthly(names,dict_of_dfs):
    d = dict_of_dfs
    for i in range(len(names)-2):
        i = names[i]
        dataset = i + '.csv'
        d[i] = pd.read_csv(dataset)
        d[i]['TIME_PERIOD'] = pd.to_datetime(d[i]['TIME_PERIOD']).dt.to_period('M')
        d[i] = d[i].set_index('TIME_PERIOD').resample('M').interpolate()
        d[i] = d[i].reset_index() ##
    d['prod_bovine'] = pd.read_csv('prod_bovine.csv')
    d['prod_chicken'] = pd.read_csv('prod_chicken.csv')
    return d

In [742]:
def merging_datasets(names, dict_data):

    df = pd.merge(dict_data[names[0]], dict_data[names[1]], how='outer', on='TIME_PERIOD')
    
    for i in range(len(names)-4):
        df1 = pd.merge(df, dict_data[names[i+2]], how='outer', on='TIME_PERIOD')
    
    df1['TIME_PERIOD'] = df1['TIME_PERIOD'].astype(str)

    df2 = pd.merge(dict_data[names[8]], dict_data[names[9]], how='outer', on='TIME_PERIOD')

    df2['TIME_PERIOD'] = pd.to_datetime(df2['TIME_PERIOD'])
    df1['TIME_PERIOD'] = pd.to_datetime(df1['TIME_PERIOD'])

    df = pd.merge(df1,df2, how='outer',on='TIME_PERIOD')

    return df

In [752]:
names = ['price_Goods_services_consumed_agri_prod15', 'price_Goods_services_contributing_agri_prod15',  'price_Goods_services_consumed_agri_prod10', 'price_Goods_services_contributing_agri_prod10', 'price_Goods_services_consumed_agri_prod05','price_Goods_services_contributing_agri_prod05', 'price_Goods_services_consumed_agri_prod00', 'price_Goods_services_contributing_agri_prod00', 'prod_bovine', 'prod_chicken']
dict_of_dfs = {}
dict_data = convert_to_monthly(names,dict_of_dfs)
df = merging_datasets(names, dict_data)

In [748]:
filepath = Path('C:/Users/t480/Desktop/University courses/ENSAE 2A/PYTHON DS/projet_python_ds//DF.csv')
df.to_csv(filepath, index=True)
#df = pd.read_csv('DF.csv', index_col=False)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,TIME_PERIOD,EU27_2020_price_Goods_services_consumed_agri_prod15,EU28_price_Goods_services_consumed_agri_prod15,FR_price_Goods_services_consumed_agri_prod15,EU27_2020_price_Goods_services_contributing_agri_prod15,EU28_price_Goods_services_contributing_agri_prod15,FR_price_Goods_services_contributing_agri_prod15,...,EU28_price_Goods_services_contributing_agri_prod00,FR_price_Goods_services_contributing_agri_prod00,EU27_2007_prod_bovine,EU27_2020_prod_bovine,EU28_prod_bovine,FR_prod_bovine,EU27_2007_prod_chicken,EU27_2020_prod_chicken,EU28_prod_chicken,FR_prod_chicken
0,0,0,0,2015-01-01,100.97,101.140000,100.830000,100.370000,100.420000,100.250000,...,100.420000,100.250000,629.59,548.70,632.72,119.82,NaN,NaN,NaN,73905.69
1,1,1,1,2015-02-01,100.70,100.866667,100.810000,100.020000,100.080000,100.096667,...,100.080000,100.096667,572.29,503.87,575.24,106.52,NaN,NaN,NaN,69263.60
2,2,2,2,2015-03-01,100.43,100.593333,100.790000,99.670000,99.740000,99.943333,...,99.740000,99.943333,652.72,587.41,656.18,135.03,NaN,NaN,NaN,74002.17
3,3,3,3,2015-04-01,100.16,100.320000,100.770000,99.320000,99.400000,99.790000,...,99.400000,99.790000,606.63,529.07,609.77,117.65,NaN,NaN,NaN,74878.53
4,4,4,4,2015-05-01,100.08,100.160000,100.383333,99.516667,99.563333,99.873333,...,99.563333,99.873333,580.77,520.83,584.22,117.08,NaN,NaN,NaN,80816.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,259,259,2014-10-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,688.47,601.01,692.19,123.38,NaN,NaN,NaN,62528.59
260,260,260,260,2014-11-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,618.05,547.71,621.48,119.74,NaN,NaN,NaN,59775.35
261,261,261,261,2014-12-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,630.05,570.57,633.92,121.86,NaN,NaN,NaN,71545.00
262,262,262,262,2021-11-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,618.07,NaN,123.13,NaN,NaN,NaN,61889.48
